# LiPF₆ + 水の溶媒和系の作成デモ（Packmol版）

このノートブックでは、LiPF₆結晶構造に水分子を充填して
溶媒和系を作成する方法を示します。

**★ Packmolを使用した高速・高品質な充填**

## 主な機能
1. LiPF₆.cifファイルの読み込み
2. 密度指定による水分子数の自動計算
3. Packmolによる最適配置（重なりなし）
4. 構造セルサイズと水充填領域を個別指定可能
5. 初期構造の保存
6. （オプション）MDシミュレーションでの緩和

## Packmolの利点
- ASEのランダム配置より高速
- 原子の重なりがない高品質な初期構造
- 大規模システムでも安定

In [ ]:
import numpy as np
import sys
from pathlib import Path
from ase.io import read, write
from ase import units

# プロジェクトのutilsをインポート（Packmol版）
sys.path.append(str(Path.cwd().parent / "LiB2_structure_ipynb"))
from utils.packmol_utils import (
    fill_box_with_packmol,
    estimate_required_molecules,
    check_packmol_command
)

# Packmolの確認
if check_packmol_command():
    print("✓ Packmolコマンドが利用可能です")
else:
    print("✗ 警告: Packmolコマンドが見つかりません")
    print("  Packmolをインストールしてください: https://m3g.github.io/packmol/")

print("✓ インポート完了")

## 1. 設定パラメータ

In [ ]:
CONFIG = {
    # 入力ファイル
    'lipf6_cif_path': '/home/jovyan/Kaori/MD/input/LiPF6.cif',
    
    # 水の密度設定
    'water_density_g_cm3': 0.9,  # 0.9 g/cm³ (緩めの初期配置)
    
    # Packmolパラメータ
    'tolerance': 2.0,  # 原子間の最小距離 (Å)
    'packmol_seed': 12345,  # 乱数シード（再現性のため）
    
    # セルサイズ設定（オプション）
    # None の場合は構造ファイルのセルサイズを使用
    'structure_cell_size': None,  # 構造全体のセルサイズ [a, b, c] または float（立方体）
    'water_fill_cell_size': None,  # 水を充填する領域のサイズ [a, b, c] または float（立方体）
    
    # 使用例:
    # 'structure_cell_size': [30.0, 30.0, 30.0],  # 30x30x30 Åのセル
    # 'water_fill_cell_size': [20.0, 20.0, 20.0],  # 20x20x20 Åの領域に水を充填
    # または
    # 'structure_cell_size': 30.0,  # 30x30x30 Åの立方体セル
    # 'water_fill_cell_size': 20.0,  # 20x20x20 Åの立方体領域に水を充填
    
    # 出力ディレクトリ
    'output_dir': 'solvation_results',
}

# 出力ディレクトリの作成
output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(exist_ok=True)

print("設定:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. LiPF₆構造の読み込み

In [ ]:
lipf6_path = CONFIG['lipf6_cif_path']

try:
    lipf6_atoms = read(lipf6_path)
    print(f"✓ LiPF₆構造を読み込みました: {lipf6_path}")
    print(f"\n構造情報:")
    print(f"  組成: {lipf6_atoms.get_chemical_formula()}")
    print(f"  原子数: {len(lipf6_atoms)}")
    print(f"  セルパラメータ: {lipf6_atoms.cell.cellpar()}")
    print(f"  体積: {lipf6_atoms.get_volume():.2f} Å³")
    
    # Li, P, F の数を数える
    symbols = lipf6_atoms.get_chemical_symbols()
    n_li = symbols.count('Li')
    n_p = symbols.count('P')
    n_f = symbols.count('F')
    print(f"\n原子数内訳:")
    print(f"  Li: {n_li}")
    print(f"  P:  {n_p}")
    print(f"  F:  {n_f}")
    print(f"  → LiPF₆分子数: {n_li} (Li基準)")
    
except FileNotFoundError:
    print(f"✗ エラー: ファイルが見つかりません: {lipf6_path}")
    raise

## 3. 必要な水分子数の見積もり

In [ ]:
# 水を充填する領域の体積を計算
if CONFIG['water_fill_cell_size'] is not None:
    # 明示的に指定された場合
    if isinstance(CONFIG['water_fill_cell_size'], (int, float)):
        water_box_size = CONFIG['water_fill_cell_size'] ** 3
    else:
        water_box_size = np.prod(CONFIG['water_fill_cell_size'])
    print(f"水充填領域: {CONFIG['water_fill_cell_size']} (指定値)")
else:
    # 構造ファイルのセルサイズを使用
    if CONFIG['structure_cell_size'] is not None:
        if isinstance(CONFIG['structure_cell_size'], (int, float)):
            water_box_size = CONFIG['structure_cell_size'] ** 3
        else:
            water_box_size = np.prod(CONFIG['structure_cell_size'])
    else:
        water_box_size = lipf6_atoms.get_volume()
    print(f"水充填領域: 構造ファイルのセルサイズと同じ")

# 水分子のモル質量
water_molar_mass = 18.015  # g/mol

# 必要な水分子数の見積もり
n_molecules_required = estimate_required_molecules(
    water_box_size, 
    water_molar_mass, 
    CONFIG['water_density_g_cm3']
)

print("\n水分子数の見積もり:")
print(f"  充填領域体積: {water_box_size:.2f} Å³ ({water_box_size * 1e-24:.3e} cm³)")
print(f"  目標密度: {CONFIG['water_density_g_cm3']} g/cm³")
print(f"  水のモル質量: {water_molar_mass:.3f} g/mol")
print(f"  必要な水分子数: {n_molecules_required}")
print(f"  必要な水原子数: {n_molecules_required * 3} (H2O x {n_molecules_required})")
print(f"\n最終構造予測:")
print(f"  既存原子数: {len(lipf6_atoms)}")
print(f"  追加原子数: {n_molecules_required * 3}")
print(f"  総原子数: {len(lipf6_atoms) + n_molecules_required * 3}")

## 4. 水分子の充填実行

この処理には数分かかる場合があります。

In [ ]:
solvated_lipf6 = fill_box_with_packmol(
    host_atoms=lipf6_atoms,
    solvent_type='H2O',
    density_g_cm3=CONFIG['water_density_g_cm3'],
    tolerance=CONFIG['tolerance'],
    seed=CONFIG['packmol_seed'],
    structure_cell_size=CONFIG['structure_cell_size'],
    water_fill_cell_size=CONFIG['water_fill_cell_size'],
    verbose=True
)

print("\n" + "=" * 60)
print("  充填完了！")
print("=" * 60)
print(f"最終組成: {solvated_lipf6.get_chemical_formula()}")
print(f"総原子数: {len(solvated_lipf6)}")

# セル情報の表示
cell_params = solvated_lipf6.cell.cellpar()
print(f"\nセル情報:")
print(f"  a={cell_params[0]:.2f} Å, b={cell_params[1]:.2f} Å, c={cell_params[2]:.2f} Å")
print(f"  α={cell_params[3]:.1f}°, β={cell_params[4]:.1f}°, γ={cell_params[5]:.1f}°")
print(f"  体積: {solvated_lipf6.get_volume():.2f} Å³")

## 5. 結果の保存

In [ ]:
# ファイル名の生成
density_str = f"{CONFIG['water_density_g_cm3']:.1f}".replace('.', '_')
output_basename = f"lipf6_h2o_d{density_str}"

# XYZ形式で保存（可視化用）
xyz_path = output_dir / f"{output_basename}_initial.xyz"
write(xyz_path, solvated_lipf6)
print(f"✓ XYZ形式で保存: {xyz_path}")

# CIF形式で保存（結晶構造情報を保持）
cif_path = output_dir / f"{output_basename}_initial.cif"
write(cif_path, solvated_lipf6)
print(f"✓ CIF形式で保存: {cif_path}")

# 統計情報をテキストファイルに保存
stats_path = output_dir / f"{output_basename}_stats.txt"
with open(stats_path, 'w') as f:
    f.write("LiPF₆ + H₂O 溶媒和系の統計情報\n")
    f.write("=" * 60 + "\n\n")
    f.write("設定パラメータ:\n")
    for key, value in CONFIG.items():
        f.write(f"  {key}: {value}\n")
    f.write("\n構造情報:\n")
    f.write(f"  組成: {solvated_lipf6.get_chemical_formula()}\n")
    f.write(f"  総原子数: {len(solvated_lipf6)}\n")
    f.write(f"  セル体積: {solvated_lipf6.get_volume():.2f} Å³\n")
    
    # 実際に追加された水分子数を計算
    n_water_atoms = len(solvated_lipf6) - len(lipf6_atoms)
    n_water_molecules = n_water_atoms // 3
    f.write(f"\n追加された水:\n")
    f.write(f"  水分子数: {n_water_molecules}\n")
    f.write(f"  水原子数: {n_water_atoms}\n")
    
    # 実際の密度
    water_mass_g = (n_water_molecules * 18.015) / units.mol
    vol_cm3 = solvated_lipf6.get_volume() * 1e-24
    actual_density = water_mass_g / vol_cm3
    f.write(f"\n実際の密度:\n")
    f.write(f"  水の密度: {actual_density:.3f} g/cm³\n")
    f.write(f"  目標密度: {CONFIG['water_density_g_cm3']} g/cm³\n")
    f.write(f"  達成率: {actual_density/CONFIG['water_density_g_cm3']*100:.1f}%\n")

print(f"✓ 統計情報を保存: {stats_path}")

print("\n" + "=" * 60)
print("  全ての処理が完了しました！")
print("=" * 60)
print(f"\n出力ファイル:")
print(f"  1. {xyz_path}")
print(f"  2. {cif_path}")
print(f"  3. {stats_path}")

## 6. （オプション）可視化

ASE GUIまたは他の可視化ツールで構造を確認できます。

In [ ]:
# Jupyter環境でプロット
try:
    from ase.visualize import view
    # view(solvated_lipf6)  # GUIが起動します（環境による）
    print("可視化: ase.visualize.view(solvated_lipf6) で確認できます")
except:
    pass

# 原子数の内訳
from collections import Counter
symbols = solvated_lipf6.get_chemical_symbols()
counts = Counter(symbols)

print("\n最終構造の原子数内訳:")
for symbol, count in sorted(counts.items()):
    print(f"  {symbol}: {count}")

## 次のステップ

1. **MDシミュレーション**: 作成した構造を`phase1c_lipf6_al_contact.ipynb`で使用
2. **緩和**: NPT緩和で密度を調整
3. **反応シミュレーション**: 高温でLiPF₆の加水分解を観測
4. **異なる密度**: `water_density_g_cm3`を変えて比較

### 推奨設定
- **初期配置**: `water_density_g_cm3 = 0.8-0.9` (緩め)
- **緩和後**: NPTで実際の密度（~1.0 g/cm³）に調整
- **最小距離**: `min_distance = 2.0-2.5 Å`

## 7. Matlantis + FIRE による構造最適化

初期配置された分子構造をMatlantisポテンシャルとFIREオプティマイザーで最適化します。

In [ ]:
from ase.optimize import FIRE
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

print("=== 構造最適化の開始 ===\n")

# Matlantis calculatorの設定
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0, model_version="v7.0.0")
calculator = ASECalculator(estimator)

# 最適化する構造のコピー
atoms_to_optimize = solvated_lipf6.copy()
atoms_to_optimize.calc = calculator

print(f"初期エネルギー計算中...")
initial_energy = atoms_to_optimize.get_potential_energy()
print(f"初期ポテンシャルエネルギー: {initial_energy:.4f} eV\n")

# FIREオプティマイザーで最適化
print("FIRE最適化を実行中...")
optimizer = FIRE(atoms_to_optimize, trajectory=str(output_dir / f"{output_basename}_optimization.traj"))
optimizer.run(fmax=0.05, steps=200)  # 力が0.05 eV/Å以下、または200ステップまで

# 最適化後のエネルギー
final_energy = atoms_to_optimize.get_potential_energy()
print(f"\n最適化完了！")
print(f"最終ポテンシャルエネルギー: {final_energy:.4f} eV")
print(f"エネルギー変化: {final_energy - initial_energy:.4f} eV\n")

# 最適化された構造を保存
optimized_path = output_dir / f"{output_basename}_optimized.xyz"
write(optimized_path, atoms_to_optimize)
print(f"✓ 最適化構造を保存: {optimized_path}")

## 8. NVTアンサンブルでの分子動力学シミュレーション

最適化された構造を使用して、NVTアンサンブル（定温定体積）で
分子動力学シミュレーションを実行します。

In [ ]:
# MD設定
MD_CONFIG = {
    'temperature': 300.0,       # 温度 (K)
    'timestep': 0.5,            # タイムステップ (fs)
    'simulation_time': 10.0,    # シミュレーション時間 (ps)
    'traj_freq': 100,           # Trajectory保存頻度（ステップ）
    'taut': 100.0,              # Berendsen熱浴の時定数 (fs)
}

print("MD設定:")
for key, value in MD_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    NVTBerendsenIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn

print("=== NVT-MD シミュレーション開始 ===\n")

# ステップ数の計算
n_steps = int(MD_CONFIG['simulation_time'] * 1000 / MD_CONFIG['timestep'])
print(f"総ステップ数: {n_steps} ({MD_CONFIG['simulation_time']} ps)\n")

# Matlantis estimator
estimator_fn = pfp_estimator_fn(
    model_version='v7.0.0',
    calc_mode='CRYSTAL_U0'
)

# MDシステムの初期化
md_system = ASEMDSystem(
    atoms_to_optimize.copy(),  # 最適化済み構造を使用
    step=0,
    time=0.0
)

# NVT インテグレータ (Berendsen熱浴)
integrator = NVTBerendsenIntegrator(
    timestep=MD_CONFIG['timestep'],
    temperature=MD_CONFIG['temperature'],
    taut=MD_CONFIG['taut'],
    fixcm=True,  # 重心の運動を固定
)

# MDシミュレーション
md_feature = MDFeature(
    integrator=integrator,
    n_run=n_steps,
    traj_file_name=str(output_dir / f"{output_basename}_md_nvt.traj"),
    traj_freq=MD_CONFIG['traj_freq'],
    estimator_fn=estimator_fn,
    show_progress_bar=True,
)

# 実行
print("MDシミュレーション実行中...\n")
md_feature(md_system)

print("\n✓ MDシミュレーション完了！")
print(f"  Trajectoryファイル: {output_dir / f'{output_basename}_md_nvt.traj'}")

## 9. MD結果の解析と可視化

In [ ]:
from ase.io import Trajectory

# Trajectoryの読み込み
traj_path = output_dir / f"{output_basename}_md_nvt.traj"
traj = Trajectory(str(traj_path))

print(f"Trajectory情報:")
print(f"  フレーム数: {len(traj)}")
print(f"  原子数: {len(traj[0])}")

# エネルギーと温度の時系列プロット
energies = []
temperatures = []
times = []

for i, atoms in enumerate(traj):
    if hasattr(atoms, 'get_potential_energy'):
        try:
            energies.append(atoms.get_potential_energy())
        except:
            energies.append(None)
    temperatures.append(atoms.get_temperature())
    times.append(i * MD_CONFIG['timestep'] * MD_CONFIG['traj_freq'] / 1000.0)  # ps

# プロット
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# ポテンシャルエネルギー
if any(e is not None for e in energies):
    valid_energies = [(t, e) for t, e in zip(times, energies) if e is not None]
    if valid_energies:
        t_valid, e_valid = zip(*valid_energies)
        axes[0].plot(t_valid, e_valid, 'b-', linewidth=1.5)
        axes[0].set_xlabel('Time (ps)', fontsize=12)
        axes[0].set_ylabel('Potential Energy (eV)', fontsize=12)
        axes[0].set_title('Energy Evolution', fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)

# 温度
axes[1].plot(times, temperatures, 'r-', linewidth=1.5)
axes[1].axhline(y=MD_CONFIG['temperature'], color='k', linestyle='--', 
                label=f"Target: {MD_CONFIG['temperature']} K")
axes[1].set_xlabel('Time (ps)', fontsize=12)
axes[1].set_ylabel('Temperature (K)', fontsize=12)
axes[1].set_title('Temperature Evolution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

# 保存
plot_path = output_dir / f"{output_basename}_md_analysis.png"
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"\n✓ 解析グラフを保存: {plot_path}")
plt.show()

print(f"\n平均温度: {np.mean(temperatures):.2f} K")
print(f"温度の標準偏差: {np.std(temperatures):.2f} K")